# Task 1: Exploratory Data Analysis (EDA)

## Financial News Sentiment Analysis - EDA

This notebook performs comprehensive EDA on the financial news dataset including:
- Descriptive Statistics
- Text Analysis and Topic Modeling
- Time Series Analysis
- Publisher Analysis


## 1. Import Libraries and Load Data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# NLP libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from wordcloud import WordCloud

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

print("Libraries imported successfully!")


In [ ]:
# Load the dataset
df = pd.read_csv('../data/raw_analyst_ratings.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()


In [ ]:
# Basic information about the dataset
print("Dataset Info:")
print(df.info())
print("\n" + "="*50)
print("\nMissing Values:")
print(df.isnull().sum())
print("\n" + "="*50)
print("\nDuplicate Rows:")
print(f"Total duplicates: {df.duplicated().sum()}")


## 2. Data Preprocessing


In [ ]:
# Drop the Unnamed: 0 column if it exists
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Extract date components for analysis
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.day_name()
df['hour'] = df['date'].dt.hour
df['date_only'] = df['date'].dt.date

print("Data preprocessing completed!")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")


## 3. Descriptive Statistics

### 3.1 Headline Length Analysis


In [ ]:
# Calculate headline length statistics
df['headline_length'] = df['headline'].str.len()
df['headline_word_count'] = df['headline'].str.split().str.len()

print("Headline Length Statistics:")
print("="*50)
print(df['headline_length'].describe())
print("\nHeadline Word Count Statistics:")
print("="*50)
print(df['headline_word_count'].describe())


In [ ]:
# Visualize headline length distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Character length distribution
axes[0].hist(df['headline_length'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Headline Character Length', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Character Count')
axes[0].set_ylabel('Frequency')
axes[0].axvline(df['headline_length'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["headline_length"].mean():.1f}')
axes[0].legend()

# Word count distribution
axes[1].hist(df['headline_word_count'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title('Distribution of Headline Word Count', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Word Count')
axes[1].set_ylabel('Frequency')
axes[1].axvline(df['headline_word_count'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["headline_word_count"].mean():.1f}')
axes[1].legend()

plt.tight_layout()
plt.show()


### 3.2 Articles per Publisher


In [ ]:
# Count articles per publisher
publisher_counts = df['publisher'].value_counts()

print("Articles per Publisher (Top 20):")
print("="*50)
print(publisher_counts.head(20))
print(f"\nTotal unique publishers: {df['publisher'].nunique()}")
print(f"Total articles: {len(df)}")


In [ ]:
# Visualize top publishers
plt.figure(figsize=(14, 8))
top_publishers = publisher_counts.head(15)
plt.barh(range(len(top_publishers)), top_publishers.values, color='steelblue')
plt.yticks(range(len(top_publishers)), top_publishers.index)
plt.xlabel('Number of Articles', fontsize=12)
plt.title('Top 15 Publishers by Article Count', fontsize=16, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


### 3.3 Publication Date Trends


In [ ]:
# Articles by year
articles_by_year = df['year'].value_counts().sort_index()
print("Articles by Year:")
print("="*50)
print(articles_by_year)


In [ ]:
# Articles by month
articles_by_month = df['month'].value_counts().sort_index()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

print("Articles by Month:")
print("="*50)
for month, count in articles_by_month.items():
    print(f"{month_names[month-1]}: {count}")


In [ ]:
# Articles by day of week
articles_by_dow = df['day_of_week'].value_counts()
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
articles_by_dow = articles_by_dow.reindex([d for d in dow_order if d in articles_by_dow.index])

print("Articles by Day of Week:")
print("="*50)
print(articles_by_dow)


In [ ]:
# Visualize temporal trends
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Articles by year
axes[0, 0].plot(articles_by_year.index, articles_by_year.values, marker='o', linewidth=2, markersize=8)
axes[0, 0].set_title('Articles Published by Year', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Number of Articles')
axes[0, 0].grid(True, alpha=0.3)

# Articles by month
axes[0, 1].bar(articles_by_month.index, articles_by_month.values, color='coral')
axes[0, 1].set_title('Articles Published by Month', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Number of Articles')
axes[0, 1].set_xticks(range(1, 13))
axes[0, 1].set_xticklabels(month_names, rotation=45)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Articles by day of week
axes[1, 0].bar(range(len(articles_by_dow)), articles_by_dow.values, color='lightgreen')
axes[1, 0].set_title('Articles Published by Day of Week', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Day of Week')
axes[1, 0].set_ylabel('Number of Articles')
axes[1, 0].set_xticks(range(len(articles_by_dow)))
axes[1, 0].set_xticklabels(articles_by_dow.index, rotation=45)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Articles by hour
articles_by_hour = df['hour'].value_counts().sort_index()
axes[1, 1].bar(articles_by_hour.index, articles_by_hour.values, color='gold')
axes[1, 1].set_title('Articles Published by Hour of Day', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Hour (UTC-4)')
axes[1, 1].set_ylabel('Number of Articles')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 4. Text Analysis and Topic Modeling

### 4.1 Common Keywords and Phrases


In [ ]:
# Prepare text for analysis
stop_words = set(stopwords.words('english'))
# Add financial domain-specific stop words
financial_stopwords = {'stock', 'stocks', 'company', 'companies', 'market', 'markets', 
                      'price', 'prices', 'trading', 'trade', 'trades', 'day', 'week', 
                      'year', 'time', 'news', 'article', 'report', 'reports'}
stop_words.update(financial_stopwords)

def extract_keywords(text):
    """Extract keywords from text"""
    if pd.isna(text):
        return []
    tokens = word_tokenize(text.lower())
    # Filter out stopwords, punctuation, and short words
    keywords = [token for token in tokens 
                if token.isalpha() and len(token) > 2 and token not in stop_words]
    return keywords

# Extract keywords from all headlines
print("Extracting keywords from headlines...")
all_keywords = []
for headline in df['headline']:
    all_keywords.extend(extract_keywords(headline))

# Count keyword frequency
keyword_counts = Counter(all_keywords)
top_keywords = keyword_counts.most_common(30)

print("\nTop 30 Keywords:")
print("="*50)
for keyword, count in top_keywords:
    print(f"{keyword:20s}: {count:6d}")


In [ ]:
# Visualize top keywords
keywords_df = pd.DataFrame(top_keywords, columns=['Keyword', 'Count'])

plt.figure(figsize=(12, 8))
plt.barh(range(len(keywords_df)), keywords_df['Count'].values, color='teal')
plt.yticks(range(len(keywords_df)), keywords_df['Keyword'].values)
plt.xlabel('Frequency', fontsize=12)
plt.title('Top 30 Keywords in Financial News Headlines', fontsize=16, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
# Create word cloud
text_for_wordcloud = ' '.join(all_keywords)

wordcloud = WordCloud(width=1200, height=600, background_color='white', 
                      max_words=100, colormap='viridis').generate(text_for_wordcloud)

plt.figure(figsize=(15, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Financial News Headlines', fontsize=18, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()


### 4.2 Extract Significant Events and Phrases


In [ ]:
# Look for significant financial events/phrases
significant_phrases = [
    'fda approval', 'price target', 'earnings', 'revenue', 'profit', 
    'merger', 'acquisition', 'ipo', 'dividend', 'split', 'upgrade', 
    'downgrade', 'analyst', 'rating', 'forecast', 'guidance', 
    '52-week high', '52-week low', 'bullish', 'bearish'
]

phrase_counts = {}
for phrase in significant_phrases:
    count = df['headline'].str.lower().str.contains(phrase, na=False).sum()
    phrase_counts[phrase] = count

phrase_df = pd.DataFrame(list(phrase_counts.items()), columns=['Phrase', 'Count'])
phrase_df = phrase_df.sort_values('Count', ascending=False)

print("Significant Financial Phrases in Headlines:")
print("="*50)
print(phrase_df.to_string(index=False))


In [ ]:
# Visualize significant phrases
plt.figure(figsize=(12, 8))
plt.barh(range(len(phrase_df)), phrase_df['Count'].values, color='purple')
plt.yticks(range(len(phrase_df)), phrase_df['Phrase'].values)
plt.xlabel('Frequency', fontsize=12)
plt.title('Significant Financial Phrases in Headlines', fontsize=16, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


## 5. Time Series Analysis

### 5.1 Publication Frequency Over Time


In [ ]:
# Create time series of daily publication frequency
daily_counts = df.groupby('date_only').size().reset_index(name='article_count')
daily_counts['date_only'] = pd.to_datetime(daily_counts['date_only'])
daily_counts = daily_counts.sort_values('date_only')

print(f"Date range: {daily_counts['date_only'].min()} to {daily_counts['date_only'].max()}")
print(f"Total days with articles: {len(daily_counts)}")
print(f"\nDaily publication statistics:")
print(daily_counts['article_count'].describe())


In [ ]:
# Plot daily publication frequency
plt.figure(figsize=(16, 6))
plt.plot(daily_counts['date_only'], daily_counts['article_count'], linewidth=1, alpha=0.7)
plt.title('Daily Publication Frequency Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Number of Articles')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Monthly aggregation for better visualization
df['year_month'] = df['date'].dt.to_period('M')
monthly_counts = df.groupby('year_month').size().reset_index(name='article_count')
monthly_counts['year_month'] = monthly_counts['year_month'].astype(str)

plt.figure(figsize=(16, 6))
plt.bar(range(len(monthly_counts)), monthly_counts['article_count'].values, 
        color='steelblue', alpha=0.7)
plt.title('Monthly Publication Frequency', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Articles')
plt.xticks(range(0, len(monthly_counts), max(1, len(monthly_counts)//20)), 
           monthly_counts['year_month'][::max(1, len(monthly_counts)//20)], 
           rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()


### 5.2 Publishing Time Analysis


In [ ]:
# Analyze publishing times
print("Publishing Time Analysis:")
print("="*50)
print(f"Most common publishing hour: {df['hour'].mode()[0]}:00")
print(f"\nArticles by hour (top 10):")
print(df['hour'].value_counts().head(10))


In [ ]:
# Visualize publishing times
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Hourly distribution
hour_counts = df['hour'].value_counts().sort_index()
axes[0].bar(hour_counts.index, hour_counts.values, color='coral', alpha=0.7)
axes[0].set_title('Article Publication by Hour of Day', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Hour (UTC-4)')
axes[0].set_ylabel('Number of Articles')
axes[0].grid(True, alpha=0.3, axis='y')

# Day of week distribution
dow_counts = df['day_of_week'].value_counts()
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_counts = dow_counts.reindex([d for d in dow_order if d in dow_counts.index])
axes[1].bar(range(len(dow_counts)), dow_counts.values, color='lightgreen', alpha=0.7)
axes[1].set_title('Article Publication by Day of Week', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Number of Articles')
axes[1].set_xticks(range(len(dow_counts)))
axes[1].set_xticklabels(dow_counts.index, rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 6. Publisher Analysis

### 6.1 Publisher Contribution Analysis


In [ ]:
# Detailed publisher statistics
publisher_stats = df.groupby('publisher').agg({
    'headline': 'count',
    'stock': 'nunique',
    'date': ['min', 'max']
}).reset_index()

publisher_stats.columns = ['publisher', 'article_count', 'unique_stocks', 'first_article', 'last_article']
publisher_stats = publisher_stats.sort_values('article_count', ascending=False)

print("Publisher Statistics (Top 20):")
print("="*70)
print(publisher_stats.head(20).to_string(index=False))


In [ ]:
# Publisher contribution percentage
total_articles = len(df)
publisher_stats['contribution_pct'] = (publisher_stats['article_count'] / total_articles * 100).round(2)

print("\nPublisher Contribution (Top 15):")
print("="*70)
top_15 = publisher_stats.head(15)[['publisher', 'article_count', 'contribution_pct']]
print(top_15.to_string(index=False))

print(f"\nTop 15 publishers account for {top_15['contribution_pct'].sum():.2f}% of all articles")


### 6.2 Publisher Domain Analysis (if email addresses are used)


In [ ]:
# Check if publishers contain email addresses
import re

def extract_domain(text):
    """Extract domain from email or text"""
    if pd.isna(text):
        return None
    # Check for email pattern
    email_pattern = r'[a-zA-Z0-9._%+-]+@([a-zA-Z0-9.-]+\.[a-zA-Z]{2,})'
    match = re.search(email_pattern, str(text))
    if match:
        return match.group(1)
    return None

# Extract domains from publishers
df['publisher_domain'] = df['publisher'].apply(extract_domain)

if df['publisher_domain'].notna().sum() > 0:
    domain_counts = df['publisher_domain'].value_counts()
    print(f"Found {len(domain_counts)} unique domains from email addresses")
    print("\nTop domains:")
    print(domain_counts.head(10))
else:
    print("No email addresses found in publisher field.")
    print("\nAnalyzing publisher names as organizations instead...")
    
    # Analyze unique publisher names
    print(f"\nTotal unique publishers: {df['publisher'].nunique()}")
    print(f"\nPublisher name patterns (sample):")
    print(df['publisher'].value_counts().head(20))


In [ ]:
# Visualize publisher distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 20 publishers
top_20_publishers = publisher_counts.head(20)
axes[0].barh(range(len(top_20_publishers)), top_20_publishers.values, color='steelblue')
axes[0].set_yticks(range(len(top_20_publishers)))
axes[0].set_yticklabels(top_20_publishers.index)
axes[0].set_xlabel('Number of Articles')
axes[0].set_title('Top 20 Publishers by Article Count', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3, axis='x')

# Publisher contribution pie chart (top 10)
top_10_publishers = publisher_counts.head(10)
others_count = publisher_counts.iloc[10:].sum()
pie_data = list(top_10_publishers.values) + [others_count]
pie_labels = list(top_10_publishers.index) + ['Others']

axes[1].pie(pie_data, labels=pie_labels, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Publisher Distribution (Top 10 + Others)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


### 6.3 Stock Coverage by Publisher


In [ ]:
# Analyze which publishers cover which stocks
publisher_stock = df.groupby(['publisher', 'stock']).size().reset_index(name='count')
publisher_stock_pivot = publisher_stock.pivot(index='publisher', columns='stock', values='count').fillna(0)

# Top publishers and their stock coverage
top_publishers_list = publisher_counts.head(10).index.tolist()
top_publishers_coverage = publisher_stock_pivot.loc[top_publishers_list]

print("Stock Coverage by Top Publishers:")
print("="*70)
print(top_publishers_coverage.to_string())


In [ ]:
# Visualize stock coverage heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(top_publishers_coverage, annot=True, fmt='.0f', cmap='YlOrRd', 
            cbar_kws={'label': 'Number of Articles'})
plt.title('Stock Coverage Heatmap by Top Publishers', fontsize=16, fontweight='bold')
plt.xlabel('Stock Symbol', fontsize=12)
plt.ylabel('Publisher', fontsize=12)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


## 7. Summary and Key Insights


In [ ]:
print("="*70)
print("EDA SUMMARY - KEY INSIGHTS")
print("="*70)

print(f"\n1. DATASET OVERVIEW:")
print(f"   - Total articles: {len(df):,}")
print(f"   - Date range: {df['date'].min()} to {df['date'].max()}")
print(f"   - Unique publishers: {df['publisher'].nunique()}")
print(f"   - Unique stocks: {df['stock'].nunique()}")

print(f"\n2. HEADLINE STATISTICS:")
print(f"   - Average headline length: {df['headline_length'].mean():.1f} characters")
print(f"   - Average word count: {df['headline_word_count'].mean():.1f} words")

print(f"\n3. PUBLISHER INSIGHTS:")
print(f"   - Top publisher: {publisher_counts.index[0]} ({publisher_counts.iloc[0]:,} articles)")
print(f"   - Top 10 publishers account for {publisher_stats.head(10)['contribution_pct'].sum():.1f}% of articles")

print(f"\n4. TEMPORAL PATTERNS:")
print(f"   - Most active year: {articles_by_year.idxmax()} ({articles_by_year.max():,} articles)")
print(f"   - Most active month: {month_names[articles_by_month.idxmax()-1]} ({articles_by_month.max():,} articles)")
print(f"   - Most active day: {articles_by_dow.index[0]} ({articles_by_dow.iloc[0]:,} articles)")
print(f"   - Peak publishing hour: {df['hour'].mode()[0]}:00")

print(f"\n5. TEXT ANALYSIS:")
print(f"   - Top keyword: '{top_keywords[0][0]}' (appears {top_keywords[0][1]:,} times)")
print(f"   - Most common phrase: '{phrase_df.iloc[0]['Phrase']}' ({phrase_df.iloc[0]['Count']:,} occurrences)")

print("\n" + "="*70)
